In [1]:
import numpy as np
import pyDR as DR
from pyDR.Project import Project
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
import pickle
import os

In [2]:
#ile_resn = [3, 13, 23, 30, 36, 44, 61]
iles = [13, 61]
test_list = ["test_001", "test_002", "test_003", "test_010", "test_011", "test_014", "test_015", "test_charm"]

In [3]:
#dir = "/localhome/mlehner/nmr/1CM8/"
dir = "/home/mlehner/pyDR/1ubq_data/"

In [4]:
def get_frame_analysis(dir, test, slice, ile):
    tf=None
    #tf=1000
    molsys=DR.MolSys('./1ubq_data/1ubq_frame.pdb', f"{dir}{test}/traj_1us_{slice}_PBC_fit.xtc",tf=tf)
    select=DR.MolSelect(molsys)

    frames=list()
    frames.append({'Type':'methylCC','Nuc':'ivl', 'resids':ile,'sigma':0})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile,'n_bonds':1,'sigma':0})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile,'n_bonds':2,'sigma':0})

    "Frames with post-process smoothing"
    frames.append({'Type':'hops_3site','Nuc':'ivl', 'resids':ile,'sigma':5})
    frames.append({'Type':'methylCC','Nuc':'ivl', 'resids':ile,'sigma':5})
    frames.append({'Type':'chi_hop','Nuc':'ivl', 'resids':ile,'n_bonds':1,'sigma':50})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile,'n_bonds':1,'sigma':50})
    frames.append({'Type':'chi_hop','Nuc':'ivl', 'resids':ile,'n_bonds':2,'sigma':50})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile,'n_bonds':2,'sigma':50})

    select.select_bond(Nuc='ivl', resids=ile)
    fr_obj=DR.Frames.FrameObj(select)
    fr_obj.tensor_frame(sel1=1,sel2=2)
    for f in frames:fr_obj.new_frame(**f)
    fr_obj.load_frames(n=-1)
    fr_obj.post_process()

    tf = molsys._traj.tf
    print(f"tf: {tf}")

    include=np.zeros([3,9],dtype=bool)
    include[0][:3]=True  
    include[1][[4,6,8]]=True
    include[2][3:]=True
    
    t=np.arange(tf>>1)*.005
    data=fr_obj.frames2data(mode='full')
    proj=Project(f"{dir}{test}/{slice}.nosync",create=True)
    for d in data:proj.append_data(d)
    proj[0].detect.r_no_opt(10)
    proj.fit(bounds=False)
    proj['no_opt'][0].detect.r_auto(5)
    proj['no_opt'].fit()
    #proj.save()
    return [fr_obj.frames2ct(include=x, mode='full') for x in include]
    

In [5]:
for test in test_list:
    for slice in range(1,5):
        for ile in iles:
            print(f"{test} {slice} {ile}", flush=True)
            try:
                out_pickle_path = f"{dir}{test}/{slice}_{ile}.p"
                if os.path.exists(out_pickle_path):
                    print(f"skipping {out_pickle_path}")
                    continue
                else:
                    print(f"creating {out_pickle_path}")
                    out = get_frame_analysis(dir, test, slice, [ile])
                    pickle.dump(out, open(out_pickle_path, "wb"))
            except Exception as e:
                print(e)
                pass

test_001 1 13
skipping /home/mlehner/pyDR/1ubq_data/test_001/1_13.p
test_001 1 61
skipping /home/mlehner/pyDR/1ubq_data/test_001/1_61.p
test_001 2 13
skipping /home/mlehner/pyDR/1ubq_data/test_001/2_13.p
test_001 2 61
skipping /home/mlehner/pyDR/1ubq_data/test_001/2_61.p
test_001 3 13
skipping /home/mlehner/pyDR/1ubq_data/test_001/3_13.p
test_001 3 61
skipping /home/mlehner/pyDR/1ubq_data/test_001/3_61.p
test_001 4 13
skipping /home/mlehner/pyDR/1ubq_data/test_001/4_13.p
test_001 4 61
skipping /home/mlehner/pyDR/1ubq_data/test_001/4_61.p
test_002 1 13
skipping /home/mlehner/pyDR/1ubq_data/test_002/1_13.p
test_002 1 61
skipping /home/mlehner/pyDR/1ubq_data/test_002/1_61.p
test_002 2 13
skipping /home/mlehner/pyDR/1ubq_data/test_002/2_13.p
test_002 2 61
skipping /home/mlehner/pyDR/1ubq_data/test_002/2_61.p
test_002 3 13
skipping /home/mlehner/pyDR/1ubq_data/test_002/3_13.p
test_002 3 61
skipping /home/mlehner/pyDR/1ubq_data/test_002/3_61.p
test_002 4 13
skipping /home/mlehner/pyDR/1ubq_d

In [6]:
"/home/mlehner/pyDR/1ubq_data/test_001/traj_1us_1_PBC_fit.xtc"

'/home/mlehner/pyDR/1ubq_data/test_001/traj_1us_1_PBC_fit.xtc'